### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [ ]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.


In [ ]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [ ]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [ ]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.policy.Policy;
import com.aerospike.client.query.IndexType;
import com.aerospike.client.query.IndexCollectionType;
import com.aerospike.client.task.IndexTask;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;
import com.aerospike.client.cdt.CTX;

In [ ]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);


In [ ]:
Key key = new Key("test", "demo", "key1");
WritePolicy wPolicy = new WritePolicy();
Bin b1 = new Bin("b1", Value.get("val1"));
Bin b2 = new Bin("b2", Value.get(2));
client.put(wPolicy, key, b1, b2);
Record record = client.get(null, key);
System.out.println("Record values are:");
System.out.println(record);

In [9]:
// convenience function to create an index - essentially a pass-through to the client API
void myCreateIndex(String nsName, String setName, String idxName, String binName, IndexType idxType, IndexCollectionType colType, CTX... ctx) {
    Policy policy = new Policy();
    try {
            IndexTask task = client.createIndex(policy, nsName, setName, idxName, binName, idxType, colType, ctx);
            task.waitTillComplete(100, 0);
        }
        catch (AerospikeException ae) {
            if (ae.getResultCode() != ResultCode.INDEX_ALREADY_EXISTS) {
                throw ae;
            }
        } 
       // System.out.format("Created index %s on ns=%s set=%s bin=%s.\n", 
       //                             idxName, nsName, setName, binName);
}
void myDropIndex(String nsName, String setName, String idxName) {
    try {
        IndexTask task = client.dropIndex(null, nsName, setName, idxName);
    }
    catch (AerospikeException ae) {
        if (ae.getResultCode() != ResultCode.INDEX_NOTFOUND) {
            throw ae;
        }
    } 
    //System.out.format("Dropped index %s.\n", idxName);
}

# Set Name Jammmer
Use up to max Set names = 4K. Takes 15+ minutes to run. Monitor progess in terminal using:
 * asadm -e "show sindex"
   
   or
   
 * aql -c "show sets"

In [10]:
String nsName = "test";
for(int i=0; i<4198; i++){ 
  String idxname = "name"+i;
  String binname = "bin"+i;
  String setName = "set"+i;
  myCreateIndex(nsName, setName, idxname, binname, IndexType.NUMERIC, IndexCollectionType.DEFAULT);
  myDropIndex(nsName, setName, idxname);  //Only max 256 sindexes can be declared.
}

# Test with new Set Name
We have used up all set names, max = 4K.

In [15]:
Key key = new Key("test", "set4197", "key1");  //New set name
WritePolicy wPolicy = new WritePolicy();
Bin b1 = new Bin("b1", Value.get("val1"));
Bin b2 = new Bin("b2", Value.get(2));
client.put(wPolicy, key, b1, b2);
Record record = client.get(null, key);
System.out.println("Record values are:");
System.out.println(record);

Record values are:
(gen:1),(exp:450832852),(bins:(b1:val1),(b2:2))


# Clean up - restart server
In the terminal, do:
* sudo service aerospike stop
* sudo rm /opt/aerospike/data/test.dat
* sudo service aerospike start